In [3]:
import sqlite3
import json
from datetime import datetime
import time

timeframe = '2015-01'
sql_transaction = []
start_row = 0
cleanup = 1000000

connection = sqlite3.connect('{}.db'.format(timeframe))
c = connection.cursor()

#store the parent_id, comment_id, the parent comment, the reply (comment)
def create_table():
    c.execute("CREATE TABLE IF NOT EXISTS parent_reply(parent_id TEXT PRIMARY KEY, comment_id TEXT UNIQUE, parent TEXT, comment TEXT, subreddit TEXT, unix INT, score INT)")

#this is to normalize the comments and to convert the newline character to a word.
def format_data(data):
    data = data.replace('\n',' newlinechar ').replace('\r',' newlinechar ').replace('"',"'")
    return data


def transaction_bldr(sql):
    global sql_transaction
    sql_transaction.append(sql)
    if len(sql_transaction) > 1000:
        c.execute('BEGIN TRANSACTION')
        for s in sql_transaction:
            try:
                c.execute(s)
            except:
                pass
        connection.commit()
        sql_transaction = []

def sql_insert_replace_comment(commentid,parentid,parent,comment,subreddit,time,score):
    try:
        sql = """UPDATE parent_reply SET parent_id = ?, comment_id = ?, parent = ?, comment = ?, subreddit = ?, unix = ?, score = ? WHERE parent_id =?;""".format(parentid, commentid, parent, comment, subreddit, int(time), score, parentid)
        transaction_bldr(sql)
    except Exception as e:
        print('s0 insertion',str(e))

def sql_insert_has_parent(commentid,parentid,parent,comment,subreddit,time,score):
    try:
        sql = """INSERT INTO parent_reply (parent_id, comment_id, parent, comment, subreddit, unix, score) VALUES ("{}","{}","{}","{}","{}",{},{});""".format(parentid, commentid, parent, comment, subreddit, int(time), score)
        transaction_bldr(sql)
    except Exception as e:
        print('s0 insertion',str(e))

def sql_insert_no_parent(commentid,parentid,comment,subreddit,time,score):
    try:
        sql = """INSERT INTO parent_reply (parent_id, comment_id, comment, subreddit, unix, score) VALUES ("{}","{}","{}","{}",{},{});""".format(parentid, commentid, comment, subreddit, int(time), score)
        transaction_bldr(sql)
    except Exception as e:
        print('s0 insertion',str(e))

# We want to make sure comments are of an acceptable length for training, and that the comment wasn't removed or deleted:
def acceptable(data):
    if len(data.split(' ')) > 1000 or len(data) < 1:
        return False
    elif len(data) > 32000:
        return False
    elif data == '[deleted]':
        return False
    elif data == '[removed]':
        return False
    else:
        return True

    
#comment_id in our database that matches another comment's parent_id, then we should match this new comment with the parent that we have already.
def find_parent(pid):
    try:
        sql = "SELECT comment FROM parent_reply WHERE comment_id = '{}' LIMIT 1".format(pid)
        c.execute(sql)
        result = c.fetchone()
        if result != None:
            return result[0]
        else: return False
    except Exception as e:
        #print(str(e))
        return False

#if our score is higher than the existing comment's score, we'd like to replace it
def find_existing_score(pid):
    try:
        sql = "SELECT score FROM parent_reply WHERE parent_id = '{}' LIMIT 1".format(pid)
        c.execute(sql)
        result = c.fetchone()
        if result != None:
            return result[0]
        else: return False
    except Exception as e:
        #print(str(e))
        return False
    
if __name__ == '__main__':
    create_table()
    #count the amount of rows we have gone through as indecates how far we are in the file 
    row_counter = 0
    #counts the amount of parent and child pairs their are 
    #meaning we have comment and reply pairs, which are training data
    paired_rows = 0
    
    # use buffering parameter because it takes to much memory so we read the file in small chunks
    #with open('J:/chatdata/reddit_data/{}/RC_{}'.format(timeframe.split('-')[0],timeframe), buffering=1000) as f:
    with open('RC_2015-01'.format(timeframe), buffering=1000) as f:
        for row in f:
            #print(row)
            #time.sleep(555)
            row_counter += 1

            if row_counter > start_row:
                try:
                    row = json.loads(row)
                    parent_id = row['parent_id'].split('_')[1]
                    body = format_data(row['body'])
                    created_utc = row['created_utc']
                    score = row['score'] 
                    comment_id = row['id']
                    
                    subreddit = row['subreddit']
                    parent_data = find_parent(parent_id)
                    
                    existing_comment_score = find_existing_score(parent_id)
                    if existing_comment_score:
                        if score > existing_comment_score:
                            if acceptable(body):
                                sql_insert_replace_comment(comment_id,parent_id,parent_data,body,subreddit,created_utc,score)
                                
                    else:
                        if acceptable(body):
                            if parent_data:
                                if score >= 2:
                                    sql_insert_has_parent(comment_id,parent_id,parent_data,body,subreddit,created_utc,score)
                                    paired_rows += 1
                            else:
                                sql_insert_no_parent(comment_id,parent_id,body,subreddit,created_utc,score)
                except Exception as e:
                    print(str(e))
                            
            if row_counter % 100000 == 0:
                print('Total Rows Read: {}, Paired Rows: {}, Time: {}'.format(row_counter, paired_rows, str(datetime.now())))


Total Rows Read: 100000, Paired Rows: 6105, Time: 2022-07-20 11:56:36.628273
Total Rows Read: 200000, Paired Rows: 13837, Time: 2022-07-20 11:56:40.504218
Total Rows Read: 300000, Paired Rows: 22238, Time: 2022-07-20 11:56:44.388622
Total Rows Read: 400000, Paired Rows: 30681, Time: 2022-07-20 11:56:48.274680
Total Rows Read: 500000, Paired Rows: 38915, Time: 2022-07-20 11:56:52.187993
Total Rows Read: 600000, Paired Rows: 46399, Time: 2022-07-20 11:56:56.110226
Total Rows Read: 700000, Paired Rows: 54145, Time: 2022-07-20 11:56:59.986516
Total Rows Read: 800000, Paired Rows: 62623, Time: 2022-07-20 11:57:03.838636
Total Rows Read: 900000, Paired Rows: 71345, Time: 2022-07-20 11:57:07.664694
Total Rows Read: 1000000, Paired Rows: 79923, Time: 2022-07-20 11:57:11.467233
Total Rows Read: 1100000, Paired Rows: 88547, Time: 2022-07-20 11:57:15.315921
Total Rows Read: 1200000, Paired Rows: 96948, Time: 2022-07-20 11:57:19.129528
Total Rows Read: 1300000, Paired Rows: 105567, Time: 2022-07-2

Total Rows Read: 10400000, Paired Rows: 914958, Time: 2022-07-20 12:03:50.594050
Total Rows Read: 10500000, Paired Rows: 923482, Time: 2022-07-20 12:03:55.172954
Total Rows Read: 10600000, Paired Rows: 931474, Time: 2022-07-20 12:03:59.734866
Total Rows Read: 10700000, Paired Rows: 939572, Time: 2022-07-20 12:04:04.703212
Total Rows Read: 10800000, Paired Rows: 948337, Time: 2022-07-20 12:04:09.046625
Total Rows Read: 10900000, Paired Rows: 957466, Time: 2022-07-20 12:04:13.373735
Total Rows Read: 11000000, Paired Rows: 967228, Time: 2022-07-20 12:04:17.729627
Total Rows Read: 11100000, Paired Rows: 976924, Time: 2022-07-20 12:04:22.192152
Total Rows Read: 11200000, Paired Rows: 986567, Time: 2022-07-20 12:04:26.583992
Total Rows Read: 11300000, Paired Rows: 996228, Time: 2022-07-20 12:04:30.965306
Total Rows Read: 11400000, Paired Rows: 1006131, Time: 2022-07-20 12:04:35.471072
Total Rows Read: 11500000, Paired Rows: 1015932, Time: 2022-07-20 12:04:39.901030
Total Rows Read: 11600000,

Total Rows Read: 20500000, Paired Rows: 1826780, Time: 2022-07-20 12:11:33.652828
Total Rows Read: 20600000, Paired Rows: 1834807, Time: 2022-07-20 12:11:38.276047
Total Rows Read: 20700000, Paired Rows: 1842874, Time: 2022-07-20 12:11:42.956459
Total Rows Read: 20800000, Paired Rows: 1850923, Time: 2022-07-20 12:11:47.549191
Total Rows Read: 20900000, Paired Rows: 1859451, Time: 2022-07-20 12:11:52.337358
Total Rows Read: 21000000, Paired Rows: 1867759, Time: 2022-07-20 12:11:57.173938
Total Rows Read: 21100000, Paired Rows: 1875590, Time: 2022-07-20 12:12:01.971724
Total Rows Read: 21200000, Paired Rows: 1883809, Time: 2022-07-20 12:12:06.770853
Total Rows Read: 21300000, Paired Rows: 1892563, Time: 2022-07-20 12:12:11.569668
Total Rows Read: 21400000, Paired Rows: 1902008, Time: 2022-07-20 12:12:16.229010
Total Rows Read: 21500000, Paired Rows: 1911477, Time: 2022-07-20 12:12:20.778400
Total Rows Read: 21600000, Paired Rows: 1920971, Time: 2022-07-20 12:12:25.452368
Total Rows Read:

Total Rows Read: 30500000, Paired Rows: 2721367, Time: 2022-07-20 12:19:25.931755
Total Rows Read: 30600000, Paired Rows: 2729207, Time: 2022-07-20 12:19:29.925567
Total Rows Read: 30700000, Paired Rows: 2735820, Time: 2022-07-20 12:19:33.868781
Total Rows Read: 30800000, Paired Rows: 2743949, Time: 2022-07-20 12:19:38.194038
Total Rows Read: 30900000, Paired Rows: 2752494, Time: 2022-07-20 12:19:42.430436
Total Rows Read: 31000000, Paired Rows: 2760634, Time: 2022-07-20 12:19:46.652471
Total Rows Read: 31100000, Paired Rows: 2769531, Time: 2022-07-20 12:19:51.048555
Total Rows Read: 31200000, Paired Rows: 2778535, Time: 2022-07-20 12:19:55.438683
Total Rows Read: 31300000, Paired Rows: 2787840, Time: 2022-07-20 12:19:59.831887
Total Rows Read: 31400000, Paired Rows: 2796722, Time: 2022-07-20 12:20:04.299316
Total Rows Read: 31500000, Paired Rows: 2804596, Time: 2022-07-20 12:20:08.921342
Total Rows Read: 31600000, Paired Rows: 2812956, Time: 2022-07-20 12:20:13.573933
Total Rows Read:

Total Rows Read: 40500000, Paired Rows: 3614435, Time: 2022-07-20 12:26:54.882851
Total Rows Read: 40600000, Paired Rows: 3622554, Time: 2022-07-20 12:26:59.614305
Total Rows Read: 40700000, Paired Rows: 3630475, Time: 2022-07-20 12:27:04.333886
Total Rows Read: 40800000, Paired Rows: 3638788, Time: 2022-07-20 12:27:08.928965
Total Rows Read: 40900000, Paired Rows: 3647436, Time: 2022-07-20 12:27:13.536731
Total Rows Read: 41000000, Paired Rows: 3656349, Time: 2022-07-20 12:27:17.952475
Total Rows Read: 41100000, Paired Rows: 3665447, Time: 2022-07-20 12:27:22.457683
Total Rows Read: 41200000, Paired Rows: 3674784, Time: 2022-07-20 12:27:27.006618
Total Rows Read: 41300000, Paired Rows: 3683975, Time: 2022-07-20 12:27:31.494057
Total Rows Read: 41400000, Paired Rows: 3693081, Time: 2022-07-20 12:27:36.141629
Total Rows Read: 41500000, Paired Rows: 3701930, Time: 2022-07-20 12:27:40.717371
Total Rows Read: 41600000, Paired Rows: 3710473, Time: 2022-07-20 12:27:45.220365
Total Rows Read:

Total Rows Read: 50500000, Paired Rows: 4519491, Time: 2022-07-20 12:34:36.697388
Total Rows Read: 50600000, Paired Rows: 4528631, Time: 2022-07-20 12:34:41.286117
Total Rows Read: 50700000, Paired Rows: 4537635, Time: 2022-07-20 12:34:45.762516
Total Rows Read: 50800000, Paired Rows: 4546629, Time: 2022-07-20 12:34:50.889205
Total Rows Read: 50900000, Paired Rows: 4555912, Time: 2022-07-20 12:34:55.480213
Total Rows Read: 51000000, Paired Rows: 4564784, Time: 2022-07-20 12:35:00.132487
Total Rows Read: 51100000, Paired Rows: 4573447, Time: 2022-07-20 12:35:04.864156
Total Rows Read: 51200000, Paired Rows: 4581484, Time: 2022-07-20 12:35:09.691417
Total Rows Read: 51300000, Paired Rows: 4589860, Time: 2022-07-20 12:35:14.338401
Total Rows Read: 51400000, Paired Rows: 4598809, Time: 2022-07-20 12:35:18.882992
Total Rows Read: 51500000, Paired Rows: 4607867, Time: 2022-07-20 12:35:23.301042
Total Rows Read: 51600000, Paired Rows: 4617293, Time: 2022-07-20 12:35:27.768806
Total Rows Read: